### Demonstration with pre trained models


In [1]:
import copy

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from utils import plot_loss_epoch

In [3]:
bidding_area = 3
model_type = "fnn"

df = pd.read_csv(f"data/NO{bidding_area}.csv")
df["timestamp"] = pd.to_datetime(df["timestamp"])
df.set_index("timestamp", inplace=True)

if model_type == "fnn":
    df["past_consumption"] = df["consumption"].shift(1)
    df.rename(
        columns={"consumption": "next_consumption", "past_consumption": "consumption"},
        inplace=True,
    )
    df = df[["temperature", "consumption", "next_consumption"]]
    df.dropna(inplace=True)

    df["time_of_day"] = df.index.hour
    df["day_of_week"] = df.index.dayofweek
    df["day_of_year"] = df.index.dayofyear


df.head()

,temperature,consumption,next_consumption,time_of_day,day_of_week,day_of_year
timestamp,,,,,,
2017-05-01 01:00:00+00:00,4.0,2674.838,2678.188,1,0,121
2017-05-01 02:00:00+00:00,3.7,2678.188,2678.341,2,0,121
2017-05-01 03:00:00+00:00,3.2,2678.341,2684.172,3,0,121
2017-05-01 04:00:00+00:00,2.7,2684.172,2733.345,4,0,121
2017-05-01 05:00:00+00:00,2.6,2733.345,2784.186,5,0,121
